In [ ]:
import copy
from datetime import datetime
import json
import math
import os
import shutil
import time
import uuid

from botocore.client import Config
import ibm_boto3
from ipywidgets import IntProgress
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
from PIL import Image
from skillsnetwork import cvstudio 
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader,random_split
from torchvision import transforms
import torchvision.models as models
from tqdm import tqdm

torch.manual_seed(0)

In [ ]:
def plot_stuff(cost, acc):    
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.plot(cost, color=color)
    ax1.set_xlabel('Iteration', color=color)
    ax1.set_ylabel('total loss', color=color)
    ax1.tick_params(axis = 'y', color=color)
    ax2 = ax1.twinx()  
    color = 'tab:blue'
    ax2.set_ylabel('accuracy', color=color)
    ax2.plot(acc, color = color)
    ax2.tick_params(axis = 'y', color = color)
    fig.tight_layout()
    plt.show()

In [ ]:
def imshow_(inp, title=None):
    'Imshow for Tensor'
    inp = inp .permute(1, 2, 0).numpy() 
    print(inp.shape)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  
    plt.show()

In [ ]:
def result(model, x, y):
    z = model(x.unsqueeze_(0))
    _, yhat = torch.max(z.data, 1)
    if yhat.item() != y:
        print(f'predicted: {yhat.item()} actual: {y}')

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
cvstudioClient = cvstudio.CVStudio()
cvstudioClient.downloadAll()

In [ ]:
percentage_train = 0.9
train_set=cvstudioClient.getDataset(
    train_test='train', percentage_train=percentage_train)
val_set=cvstudioClient.getDataset(
    train_test='test', percentage_train=percentage_train)

In [ ]:
i = 0
for x, y in val_set:
    imshow_(x, f'y = {y.item()}')
    i += 1
    if i == 3:
        break

In [ ]:
n_epochs = 10
batch_size = 32
lr = 0.000001
momentum = 0.9
lr_scheduler = True
base_lr = 0.001
max_lr = 0.01

In [ ]:
def train_model(
        model, 
        train_loader,
        validation_loader, 
        criterion, 
        optimizer, 
        n_epochs,
        print_=True):
    loss_list = []
    accuracy_list = []
    correct = 0
    n_test = len(val_set)
    accuracy_best = 0
    best_model_wts = copy.deepcopy(model.state_dict())
    print('The first epoch should take several minutes')
    for epoch in tqdm(range(n_epochs)):        
        loss_sublist = []
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            model.train() 
            z = model(x)
            loss = criterion(z, y)
            loss_sublist.append(loss.data.item())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print(f'epoch {epoch} done')
        scheduler.step()    
        loss_list.append(np.mean(loss_sublist))
        correct = 0
        for x_test, y_test in validation_loader:
            x_test, y_test = x_test.to(device), y_test.to(device)
            model.eval()
            z = model(x_test)
            _, yhat = torch.max(z.data, 1)
            correct += (yhat == y_test).sum().item()
        accuracy = correct / n_test
        accuracy_list.append(accuracy)
        if accuracy > accuracy_best:
            accuracy_best = accuracy
            best_model_wts = copy.deepcopy(model.state_dict())
        if print_:
            print('learning rate', optimizer.param_groups[0]['lr'])
            print(
                f'The mean validation cost for each epoch {epoch + 1}: '
                f'{np.mean(loss_sublist)}')
            print(
                f'The validation accuracy for epoch {epoch + 1}: '
                f'{accuracy}') 
    model.load_state_dict(best_model_wts)    
    return accuracy_list, loss_list, model

In [ ]:
model = models.resnet18(pretrained=True)

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
n_classes=train_set.n_classes
n_classes

In [ ]:
model.fc = nn.Linear(512, n_classes)
model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
train_loader = torch.utils.data.DataLoader(
    dataset=train_set, batch_size=batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(
    dataset=val_set, batch_size=1)
optimizer = torch.optim.SGD(
    model.parameters(), lr=lr, momentum=momentum)

In [ ]:
if lr_scheduler:
    scheduler = torch.optim.lr_scheduler.CyclicLR(
        optimizer, 
        base_lr=0.001, 
        max_lr=0.01,
        step_size_up=5,
        mode='triangular2')

In [ ]:
start_datetime = datetime.now()
start_time=time.time()
accuracy_list, loss_list, model = train_model(
    model, 
    train_loader, 
    validation_loader, 
    criterion, 
    optimizer, 
    n_epochs = n_epochs)
end_datetime = datetime.now()
current_time = time.time()
elapsed_time = current_time - start_time
print('elapsed time:', elapsed_time )

In [ ]:
arameters = {
    'epochs': n_epochs,
    'learningRate': lr,
    'momentum': momentum,
    'percentage used training': percentage_train,
    'learningRatescheduler': {
        'lr_scheduler': lr_scheduler,
        'base_lr': base_lr, 
        'max_lr' :max_lr}}
result = cvstudioClient.report(
    started=start_datetime,
    completed=end_datetime,
    parameters=parameters,
    accuracy={'accuracy': accuracy_list, 'loss': loss_list})
if result.ok:
    print(
        'Congratulations your results have been reported back to CV '
        'Studio!')

In [ ]:
torch.save(model.state_dict(), 'model.pt')
result = cvstudioClient.uploadModel(
    'model.pt', {'numClasses': n_classes})

In [ ]:
plot_stuff(loss_list, accuracy_list)

In [ ]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, n_classes)
model.load_state_dict(torch.load('model.pt'))
model.eval()